# Hydropower Simulations with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways.
This example demonstrates a few of the options for modeling hydropower generation.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using PowerSystemCaseBuilder

### Data management packages

In [3]:
using Dates
using DataFrames

### Optimization packages

In [4]:
using Cbc # solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.05)
odir = mktempdir() #tmpdir for build steps

"/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_bvsMTX"

### Data
PowerSystemCaseBuilder links to some meaningless test data that is suitable for this example.
We can load three systems of different resolution for the examples here:

In [5]:
c_sys5_hy_wk = build_system(SIIPExampleSystems, "5_bus_hydro_wk_sys")
c_sys5_hy_uc = build_system(SIIPExampleSystems, "5_bus_hydro_uc_sys")
c_sys5_hy_ed = build_system(SIIPExampleSystems, "5_bus_hydro_ed_sys")

[ Info: Deserializing with InMemoryTimeSeriesStorage is currently not supported. Using HDF
[ Info: Loaded time series from storage file existing=5_bus_hydro_wk_sys_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_Je4VQV
[ Info: Deserializing with InMemoryTimeSeriesStorage is currently not supported. Using HDF
[ Info: Loaded time series from storage file existing=5_bus_hydro_uc_sys_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_aHKhuP
[ Info: Deserializing with InMemoryTimeSeriesStorage is currently not supported. Using HDF
[ Info: Loaded time series from storage file existing=5_bus_hydro_ed_sys_time_series_storage.h5 new=/var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/jl_0VkgcT


,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
4,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
5,HydroEnergyReservoir,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
6,HydroPumpedStorage,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
7,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
8,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
9,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3


This line just overloads JuMP printing to remove double underscores added by PowerSimulations.jl

In [6]:
PSI.JuMP._wrap_in_math_mode(str) = "\$\$ $(replace(str, "__"=>"")) \$\$"

## Two PowerSimulations features determine hydropower representation.
There are two principal ways that we can customize hydropower representation in
PowerSimulations. First, we can play with the formulation applied to hydropower generators
using the `DeviceModel`. We can also adjust how simulations are configured to represent
different decision making processes and the information flow between those processes.

### Hydropower `DeviceModel`s

First, the assignment of device formulations to particular device types gives us control
over the representation of devices. This is accomplished by defining `DeviceModel`
instances. For hydro power representations, we have two available generator types in
PowerSystems:

In [7]:
print_tree(HydroGen)

HydroGen
├─ HydroDispatch
├─ HydroEnergyReservoir
└─ HydroPumpedStorage


And in PowerSimulations, we have several available formulations that can be applied to
the hydropower generation devices:

In [8]:
print_tree(PSI.AbstractHydroFormulation)

AbstractHydroFormulation
├─ AbstractHydroDispatchFormulation
│  ├─ HydroDispatchRunOfRiver
│  └─ AbstractHydroReservoirFormulation
│     ├─ HydroDispatchPumpedStorage
│     ├─ HydroDispatchPumpedStoragewReservation
│     ├─ HydroDispatchReservoirBudget
│     └─ HydroDispatchReservoirStorage
└─ AbstractHydroUnitCommitment
   ├─ HydroCommitmentReservoirBudget
   ├─ HydroCommitmentReservoirStorage
   └─ HydroCommitmentRunOfRiver


Let's see what some of the different combinations create. First, let's apply the
`HydroDispatchRunOfRiver` formulation to the `HydroEnergyReservoir` generators, and the
`FixedOutput` formulation to `HydroDispatch` generators.
 - The `FixedOutput` formulation just acts
like a load subtractor, forcing the system to accept it's generation.
 - The `HydroDispatchRunOfRiver` formulation represents the the energy flowing out of
a reservoir. The model can choose to produce power with that energy or just let it spill by.

In [9]:
template = OperationsProblemTemplate()
set_device_model!(template, HydroEnergyReservoir, HydroDispatchRunOfRiver)
set_device_model!(template, HydroDispatch, FixedOutput)
set_device_model!(template, PowerLoad, StaticPowerLoad)

op_problem = OperationsProblem(template, c_sys5_hy_uc, horizon = 2)
build!(op_problem, output_dir = odir)

BuildStatus.BUILT = 0

Now we can see the resulting JuMP model:

In [10]:
op_problem.internal.optimization_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 4
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

The first two constraints are the power balance constraints that require the generation
from the controllable `HydroEnergyReservoir` generators to be equal to the load (flat 10.0 for all time periods)
minus the generation from the `HydroDispatch` generators [1.97, 1.983, ...]. The 3rd and 4th
constraints limit the output of the `HydroEnergyReservoir` generator to the limit defined by the
`max_activepwoer` time series. And the last 4 constraints are the lower and upper bounds of
the `HydroEnergyReservoir` operating range.

Next, let's apply the `HydroDispatchReservoirBudget` formulation to the `HydroEnergyReservoir` generators.

In [11]:
template = OperationsProblemTemplate()
set_device_model!(template, HydroEnergyReservoir, HydroDispatchReservoirBudget)
set_device_model!(template, PowerLoad, StaticPowerLoad)

op_problem = PSI.OperationsProblem(template, c_sys5_hy_uc, horizon = 2)
build!(op_problem, output_dir = odir)

BuildStatus.BUILT = 0

And, the resulting JuMP model:

In [12]:
op_problem.internal.optimization_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 4
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

Finally, let's apply the `HydroDispatchReservoirStorage` formulation to the `HydroEnergyReservoir` generators.

In [13]:
template = OperationsProblemTemplate()
set_device_model!(template, HydroEnergyReservoir, HydroDispatchReservoirStorage)
set_device_model!(template, PowerLoad, StaticPowerLoad)

op_problem = PSI.OperationsProblem(template, c_sys5_hy_uc, horizon = 24)
build!(op_problem, output_dir = odir)

BuildStatus.BUILT = 0

In [14]:
op_problem.internal.optimization_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 240
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 120 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 192 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 144 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

### Multi-Stage `SimulationSequence`
The purpose of a multi-stage simulation is to represent scheduling decisions consistently
with the time scales that govern different elements of power systems.

#### Multi-Day to Daily Simulation:
In the multi-day model, we'll use a really simple representation of all system devices
so that we can maintain computational tractability while getting an estimate of system
requirements/capabilities.

In [15]:
template_md = OperationsProblemTemplate()
set_device_model!(template_md, ThermalStandard, ThermalDispatch)
set_device_model!(template_md, PowerLoad, StaticPowerLoad)
set_device_model!(template_md, HydroEnergyReservoir, HydroDispatchReservoirStorage)

For the daily model, we can increase the modeling detail since we'll be solving shorter
problems.

In [16]:
template_da = OperationsProblemTemplate()
set_device_model!(template_da, ThermalStandard, ThermalDispatch)
set_device_model!(template_da, PowerLoad, StaticPowerLoad)
set_device_model!(template_da, HydroEnergyReservoir, HydroDispatchReservoirStorage)

In [17]:
problems = SimulationProblems(
    MD = OperationsProblem(
        template_md,
        c_sys5_hy_wk,
        optimizer = solver,
        system_to_file = false,
    ),
    DA = OperationsProblem(
        template_da,
        c_sys5_hy_uc,
        optimizer = solver,
        system_to_file = false,
    ),
)

SimulationProblems(OrderedCollections.OrderedDict{Symbol, OperationsProblem}(:MD => OperationsProblem()
, :DA => OperationsProblem()
), [:MD, :DA])

This builds the sequence and passes the the energy dispatch schedule for the `HydroEnergyReservoir`
generator from the "MD" problem to the "DA" problem in the form of an energy limit over the
synchronized periods.

In [18]:
sequence = SimulationSequence(
    problems = problems,
    feedforward_chronologies = Dict(("MD" => "DA") => Synchronize(periods = 2)),
    intervals = Dict("MD" => (Hour(48), Consecutive()), "DA" => (Hour(24), Consecutive())),
    feedforward = Dict(
        ("DA", :devices, :HydroEnergyReservoir) => IntegralLimitFF(
            variable_source_problem = PSI.ACTIVE_POWER,
            affected_variables = [PSI.ACTIVE_POWER],
        ),
    ),
    cache = Dict(("MD", "DA") => StoredEnergy(HydroEnergyReservoir, PSI.ENERGY)),
    ini_cond_chronology = IntraProblemChronology(),
);

In [19]:
sim = Simulation(
    name = "hydro",
    steps = 1,
    problems = problems,
    sequence = sequence,
    simulation_folder = odir,
)

build!(sim)

┌ Info: 
│  ──────────────────────────────────────────────────────────────────────────────
│                                        Time                   Allocations      
│                                ──────────────────────   ───────────────────────
│        Tot / % measured:            176ms / 100%            16.2MiB / 100%     
│ 
│  Section               ncalls     time   %tot     avg     alloc   %tot      avg
│  ──────────────────────────────────────────────────────────────────────────────
│  Build Simulation           1    176ms   100%   176ms   16.2MiB  100%   16.2MiB
│    Build Problems           1    173ms  98.4%   173ms   16.2MiB  100%   16.2MiB
│      Problem DA             1    151ms  86.0%   151ms   14.4MiB  88.9%  14.4MiB
│        ThermalStandard      1    102ms  57.7%   102ms   2.08MiB  12.8%  2.08MiB
│        HydroEnergyR...      1   7.49ms  4.25%  7.49ms   1.18MiB  7.27%  1.18MiB
│        PowerLoad            1   3.99ms  2.27%  3.99ms    185KiB  1.11%   185KiB
│   

BuildStatus.BUILT = 0

We can look at the "MD" Model

In [20]:
sim.problems["MD"].internal.optimization_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 52
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 20 constraints
`JuMP.AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 10 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 10 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 44 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can look at the "DA" model

In [21]:
sim.problems["DA"].internal.optimization_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 624
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 240 constraints
`JuMP.AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 120 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 122 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 576 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 528 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can execute the simulation by running the following command
```julia
execute!(sim)
```

#### 3-Stage Simulation:

In [22]:
transform_single_time_series!(c_sys5_hy_wk, 2, Hour(24)) # TODO fix PSI to enable longer intervals of stage 1

problems = SimulationProblems(
    MD = OperationsProblem(
        template_md,
        c_sys5_hy_wk,
        optimizer = solver,
        system_to_file = false,
    ),
    DA = OperationsProblem(
        template_da,
        c_sys5_hy_uc,
        optimizer = solver,
        system_to_file = false,
    ),
    ED = OperationsProblem(
        template_da,
        c_sys5_hy_ed,
        optimizer = solver,
        system_to_file = false,
    ),
)

sequence = SimulationSequence(
    problems = problems,
    feedforward_chronologies = Dict(
        ("MD" => "DA") => Synchronize(periods = 2),
        ("DA" => "ED") => Synchronize(periods = 24),
    ),
    intervals = Dict(
        "MD" => (Hour(24), Consecutive()),
        "DA" => (Hour(24), Consecutive()),
        "ED" => (Hour(1), Consecutive()),
    ),
    feedforward = Dict(
        ("DA", :devices, :HydroEnergyReservoir) => IntegralLimitFF(
            variable_source_problem = PSI.ACTIVE_POWER,
            affected_variables = [PSI.ACTIVE_POWER],
        ),
        ("ED", :devices, :HydroEnergyReservoir) => IntegralLimitFF(
            variable_source_problem = PSI.ACTIVE_POWER,
            affected_variables = [PSI.ACTIVE_POWER],
        ),
    ),
    cache = Dict(("MD", "DA") => StoredEnergy(HydroEnergyReservoir, PSI.ENERGY)),
    ini_cond_chronology = IntraProblemChronology(),
);

In [23]:
sim = Simulation(
    name = "hydro",
    steps = 1,
    problems = problems,
    sequence = sequence,
    simulation_folder = odir,
)

Simulation()


In [24]:
build!(sim)

┌ Info: 
│  ──────────────────────────────────────────────────────────────────────────────
│                                        Time                   Allocations      
│                                ──────────────────────   ───────────────────────
│        Tot / % measured:            173ms / 100%            23.9MiB / 100%     
│ 
│  Section               ncalls     time   %tot     avg     alloc   %tot      avg
│  ──────────────────────────────────────────────────────────────────────────────
│  Build Simulation           1    173ms   100%   173ms   23.9MiB  100%   23.9MiB
│    Build Problems           1    170ms  98.5%   170ms   23.7MiB  99.2%  23.7MiB
│      Problem ED             1   87.3ms  50.5%  87.3ms   7.50MiB  31.4%  7.50MiB
│        HydroEnergyR...      1   6.42ms  3.71%  6.42ms    649KiB  2.65%   649KiB
│        ThermalStandard      1   6.10ms  3.53%  6.10ms   1.13MiB  4.72%  1.13MiB
│        PowerLoad            1   3.21ms  1.85%  3.21ms    113KiB  0.46%   113KiB
│   

BuildStatus.BUILT = 0

We can look at the "MD" Model

In [25]:
sim.problems["MD"].internal.optimization_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 52
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 20 constraints
`JuMP.AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 10 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 10 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 44 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can look at the "DA" model

In [26]:
sim.problems["DA"].internal.optimization_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 624
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 240 constraints
`JuMP.AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 120 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 122 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 576 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 528 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can look at the "ED" model

In [27]:
sim.problems["ED"].internal.optimization_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 312
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 120 constraints
`JuMP.AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 60 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 62 constraints
`JuMP.VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 288 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 264 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can execute the simulation by running the following command
```julia
execute!(sim)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*